# Etude de l'impact du choix du LLM sur les persformances
Présentation de ce que l'on veut faire ici.

In [1]:
from google import genai
from google.oauth2 import service_account
import hashlib
import requests
from pathlib import Path
import tomllib
import pandas as pd
from io import StringIO
import re
from typing import Dict, Any
from tqdm import tqdm

## 1. Initialisation
Nous commençons tout d'abord par importer les bibliothèques nécessaires et initialiser le client permettant d'accéder à un LLM via VertexAI.

In [2]:

# 1. Path jusqu'au fichier de clé JSON
KEY_PATH = "projet-tutore-uga-98c5d56c8a8e.json"

# 2. Credentials object
SCOPES = ["https://www.googleapis.com/auth/cloud-platform"]
CREDENTIALS = service_account.Credentials.from_service_account_file(
    KEY_PATH,
    scopes=SCOPES
    )

# 3.Paramètres du projet
PROJECT_ID = "projet-tutore-uga"
REGION = "global"

# 4. Initialisation du client GenAI
client = genai.Client(
    vertexai=True, project=PROJECT_ID, location=REGION, credentials=CREDENTIALS
)

# 5. Modèle
MODEL_ID = "gemini-3-pro-preview"

## 2. Jeu de données
Récupération des données

In [3]:
# Define the URL and expected SHA-1 checksum
url = "https://www.data.gouv.fr/fr/datasets/r/bc085888-e6bd-445d-b3f4-632190c29e3f"
expected_sha1 = "90540350af64eb61f8a9823c83468934b19634c1"

# Define the target directory and file path
data_dir = Path("../data")
data_dir.mkdir(parents=True, exist_ok=True)
file_path = data_dir / "01_251021_AvisFiscalite.csv"

# Check if the file already exists
if file_path.exists():
    print(f"File already exists: {file_path}")
else:
    # Download the file if it doesn't exist
    print(f"Downloading file: {file_path}")
    response = requests.get(url)
    response.raise_for_status()  # Raise an error for bad HTTP responses
    file_path.write_bytes(response.content)

    # Verify the SHA-1 checksum
    sha1 = hashlib.sha1()
    with file_path.open("rb") as f:
        while chunk := f.read(8192):
            sha1.update(chunk)
    calculated_sha1 = sha1.hexdigest()
    if calculated_sha1 == expected_sha1:
        print("SHA-1 checksum verified successfully.")
    else:
        print(f"SHA-1 checksum mismatch! Expected: {expected_sha1}, Got: {calculated_sha1}")

File already exists: ..\data\01_251021_AvisFiscalite.csv


In [4]:
df = pd.read_csv(file_path, sep=",")
col_name = "QUXVlc3Rpb246MTYz - Que faudrait-il faire pour rendre la fiscalité plus juste et plus efficace ?"
df_contrib = df[['authorId', col_name]].rename(
    {col_name:'contribution'},
    axis=1)
df_contrib = df_contrib.dropna(subset=['contribution'])

C:\Users\garan\AppData\Local\Temp\ipykernel_13492\299903361.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep=",")


In [5]:
df_contrib.head()

,authorId,contribution
3,VXNlcjpjNDY0ZjllMy0xZDk4LTExZTktOTRkMi1mYTE2M2...,Repartir les richesses. suppression de la tax...
4,VXNlcjo3MDdkM2IzOC0xZDYxLTExZTktOTRkMi1mYTE2M2...,"Les droits soient automatiques, comme nos devo..."
7,VXNlcjoxZTNlOTExYi0xZTIwLTExZTktOTRkMi1mYTE2M2...,réduire drastiquement la fraude fiscale. Impos...
8,VXNlcjo1ODljMWRiMy0xZDVhLTExZTktOTRkMi1mYTE2M2...,diminuer le taux de prelevement pour les retra...
9,VXNlcjo0OTUzNmNmYy0xZTIwLTExZTktOTRkMi1mYTE2M2...,TOUT FRANÇAIS DEVRA PAYER L’IMPÔT QU’IL SOIT D...


## 3. Extraction avec le nouveau LLM

Fonctions utilitaires

In [6]:
class LLMBadCSV(Exception):
    pass

# Fonction pour nettoyer les balises de code et autres ajouts indésirables
def strip_code_fences(s: str) -> str:
    # Supprime les balises de code Markdown
    s = s.strip()
    s = re.sub(r"^```[a-zA-Z]*\s*", "", s)
    s = re.sub(r"\s*```$", "", s)
    # Supprime tout préfixe avant "CSV:" si ça arrive
    idx = s.find("CSV:")
    if idx != -1:
        s = s[idx:]
    return s

# Fonction pour extraire le bloc CSV du texte retourné par le LLM
def extract_csv_block(s: str) -> str:
    s = strip_code_fences(s)
    # On vérifie si la sortie commence par "CSV:" comme on s'y attendd'après le prompt
    if s.startswith("CSV:"):
        return s[len("CSV:"):] # On retourne tout ce qui suit "CSV:"
    # Sinon, on tente de récupérer les lignes à partir de l'entête demandée dans le prompt
    m = re.search(r"(?mi)^description,type,syntax,semantic\s*$", s)
    if m:
        return s[m.start():] # On retourne tout à partir de l'entête
    return s  # Sinon on retourne tout le texte

# Fonction pour normaliser une ligne du CSV si les contraintes ne sont pas respectées
def normalize_row(row: Dict[str, Any]) -> Dict[str, Any]:
    # Normalise les valeurs attendues
    row["type"] = str(row.get("type","")).strip().lower()
    row["syntax"] = str(row.get("syntax","")).strip().lower()
    row["semantic"] = str(row.get("semantic","")).strip().lower()

    # Contraintes
    type_ok = {"statement", "proposition"}
    syntax_ok = {"positive", "negative"}
    semantic_ok = {"positive", "negative", "neutral"}

    # Normalisation si besoin
    if row["type"] not in type_ok:
        row["type"] = "statement" 
    if row["syntax"] not in syntax_ok:
        row["syntax"] = "positive"
    if row["semantic"] not in semantic_ok:
        row["semantic"] = "neutral"
    return row

# Fonction pour parser le CSV retourné par le LLM en DataFrame pandas
def parse_llm_csv(csv_text: str) -> pd.DataFrame:
    csv_text = csv_text.strip()
    if not csv_text.lower().startswith("description,type,syntax,semantic"):
        # Parfois le modèle met des espaces, on nettoie la première ligne
        lines = csv_text.splitlines()
        if lines:
            header = lines[0].replace(" ", "")
            if header.lower() == "description,type,syntax,semantic":
                lines[0] = "description,type,syntax,semantic"
                csv_text = "\n".join(lines)

    try:
        df = pd.read_csv(StringIO(csv_text), dtype=str, keep_default_na=False)
    except Exception as e:
        raise LLMBadCSV(f"CSV illisible: {e}")

    # Colonnes minimales
    expected_cols = ["description", "type", "syntax", "semantic"]
    missing = [c for c in expected_cols if c not in df.columns]
    if missing:
        raise LLMBadCSV(f"Colonnes manquantes: {missing}")

    # Normalisation
    df = df[expected_cols].copy()
    df = df.apply(lambda r: pd.Series(normalize_row(r.to_dict())), axis=1)
    return df

def build_message(text: str) -> str:
    message = f"""
    But: extraire les idées principales DISTINCTES d'un texte pour analyse.

    Règles:
    1. N'utiliser QUE le contenu de la CONTRIBUTION.
    2. Extraire la liste des idées DISTINCTES et PRINCIPALES.
    - Chaque idée = une phrase claire, autonome, reformulée si nécessaire.
    3. Pour CHAQUE idée, annoter:
    - type: "statement" (constat) OU "proposition" (suggestion/recommandation/objectif).
    - syntax: "negative" si la phrase contient une négation explicite (ex.: "ne", "n'", "ne pas", "ne plus", "non"), sinon "positive".
    - semantic: "positive", "negative" ou "neutral" (valence sémantique).
    4. Sortie STRICTEMENT en CSV avec entête EXACTE:
    CSV:description,type,syntax,semantic
    - Délimiteur: virgule.
    - Chaque description entre guillemets doubles.
    - Échapper tout guillemet interne par duplication (ex.: ""chat"").
    - NE RIEN AJOUTER d'autre (pas de texte avant/après, pas de code fences).
    - Pas de lignes vides.

    Exemple: "Les chats retombent sur leurs pattes. Les chats n'ont pas neuf vies. Il faut mieux prendre soin des chats pour prolonger leur vie." 
    CSV:description,type,syntax,semantic
    "Les chats retombent sur leurs pattes",statement,positive,neutral
    "Les chats n'ont pas neuf vies",statement,negative,negative
    "Il faut mieux prendre soin des chats pour prolonger leur vie",proposition,positive,positive

    CONTRIBUTION:
    {text}
    """
    
    return message

# Fonction principale pour appeler le LLM et obtenir un DataFrame avec les idées extraites
def call_llm_return_df(text: str) -> pd.DataFrame:
    messages = build_message(text)
    response = client.models.generate_content(model=MODEL_ID, contents=messages)
    raw = response.text
    csv_block = extract_csv_block(raw)
    return parse_llm_csv(csv_block)

In [7]:
# Fonction pour itérer sur les contributions et extraire les idées en utilisant le LLM
def extract_ideas_from_df(df_contrib: pd.DataFrame,
                          text_col: str = "contribution",
                          id_col: str = "authorId") -> pd.DataFrame:
    rows = []

    for i, row in tqdm(df_contrib.iterrows(), total=len(df_contrib), desc="LLM extraction"):
        text = str(row[text_col]).strip()
        auth = row[id_col]
        if not text:
            continue
        try:
            ideas_df = call_llm_return_df(text)
        except Exception as e:
            # On enregistre une ligne "échec" minimale pour traçabilité
            ideas_df = pd.DataFrame([{
                "description": f"[PARSE_FAIL] {str(e)[:200]}",
                "type": "statement",
                "syntax": "positive",
                "semantic": "neutral"
            }])

        # Ajoute le contexte
        ideas_df = ideas_df.copy()
        ideas_df.insert(0, "authorId", auth)
        ideas_df.insert(1, "contrib_index", i)
        ideas_df.insert(2, "contribution", text)

        # Concatène les idées extraites
        concatenated_ideas = " || ".join(ideas_df["description"].tolist())
        rows.append({
            "authorId": auth,
            "contrib_index": i,
            "contribution": text,
            "ideas": concatenated_ideas,
            "type": "statement",  # Garder les valeurs par défaut pour type, syntax et semantic
            "syntax": "positive",
            "semantic": "neutral"
        })

    if not rows:
        return pd.DataFrame(columns=["authorId", "contrib_index", "contribution", "ideas", "type", "syntax", "semantic"])
    out = pd.DataFrame(rows)
    return out

In [8]:
out_path = data_dir / "extraction_idees_principales_Gemini.csv"
if out_path.exists():
    msg = f"L'extraction a déjà été réalisée, {out_path}"
    print(msg)
else:
    result = extract_ideas_from_df(df_contrib[0:200])
    out_path.parent.mkdir(parents=True, exist_ok=True)
    result.to_csv(out_path, index=False)
    print(f"Extraction saved to {out_path}")

LLM extraction:   0%|          | 0/200 [00:00<?, ?it/s]

LLM extraction: 100%|██████████| 200/200 [1:43:47<00:00, 31.14s/it]  

Extraction saved to ..\data\extraction_idees_principales_Gemini.csv


In [9]:
result.head()

,authorId,contrib_index,contribution,ideas,type,syntax,semantic
0,VXNlcjpjNDY0ZjllMy0xZDk4LTExZTktOTRkMi1mYTE2M2...,3,Repartir les richesses. suppression de la tax...,Il faut répartir les richesses || Il faut supp...,statement,positive,neutral
1,VXNlcjo3MDdkM2IzOC0xZDYxLTExZTktOTRkMi1mYTE2M2...,4,"Les droits soient automatiques, comme nos devo...","Les droits doivent être automatiques, tout com...",statement,positive,neutral
2,VXNlcjoxZTNlOTExYi0xZTIwLTExZTktOTRkMi1mYTE2M2...,7,réduire drastiquement la fraude fiscale. Impos...,Il faut réduire drastiquement la fraude fiscal...,statement,positive,neutral
3,VXNlcjo1ODljMWRiMy0xZDVhLTExZTktOTRkMi1mYTE2M2...,8,diminuer le taux de prelevement pour les retra...,Diminuer le taux de prélèvement pour les retra...,statement,positive,neutral
4,VXNlcjo0OTUzNmNmYy0xZTIwLTExZTktOTRkMi1mYTE2M2...,9,TOUT FRANÇAIS DEVRA PAYER L’IMPÔT QU’IL SOIT D...,Tout Français devra payer l'impôt qu'il soit d...,statement,positive,neutral


## 4. Notation humaine, hallucinations et idées séparées
De nouveau, évaluer les 200 extractions et flag la présence d'hallucinations et d'idées séparées

## 5. Evaluation sur la pipeline

## 6. Global

Compte-rendu global